In [1]:
import numpy as np
import pandas as pd

## Input Destination

You need to input the destination folder and give it a file name at the end before running the full code.

In [51]:
# Write pathway and file name for overall file

overall_final = output_final = "C:\\Users\\derek.castleman\\Desktop\\CFA Data\\CFA Data 23-24\\CFA 2\\CFA_Overall_Marco.xlsx"

standards_final = output_final = "C:\\Users\\derek.castleman\\Desktop\\CFA Data\\CFA Data 23-24\\CFA 2\\CFA_Standards_Marco.xlsx"

In [2]:
marco = input('Pathway of raw Marco learning data (get rid of quotations):   ')
marco = pd.read_excel(marco)

Pathway of raw Marco learning data (get rid of quotations):   C:\Users\derek.castleman\Desktop\CFA Data\CFA Data 23-24\CFA 2\Marco Learning CFA\Results\All Data.xlsx


In [3]:
marco

,School,SA ID,Assignment,Educator Name,Submitted on,student_id,Student Name,Holistic Score,Total,Holistic Score Rationale,reason_skipped
0,Wonderful College Preparatory Academy - Writin...,8195,6th Grade - Benchmark #1 (Radios Essay) - Fal...,Joie Garcia,2023-12-09 11:45:00,1095032,"Acol, Leo",3.0,3.0,Hi Leo! You scored a 3 for your essay because ...,NaN
1,Wonderful College Preparatory Academy - Writin...,8195,6th Grade - Benchmark #1 (Radios Essay) - Fal...,Joie Garcia,2023-12-09 11:45:00,1095456,"Albarran, Arianny",NaN,NaN,NaN,Student work is missing
2,Wonderful College Preparatory Academy - Writin...,8195,6th Grade - Benchmark #1 (Radios Essay) - Fal...,Joie Garcia,2023-12-09 11:45:00,1095921,"Baez, Luis",3.0,3.0,Hi Luis! You received 3 points for this essay ...,NaN
3,Wonderful College Preparatory Academy - Writin...,8195,6th Grade - Benchmark #1 (Radios Essay) - Fal...,Joie Garcia,2023-12-09 11:45:00,1095465,"Basurto Jr, Israel",2.0,2.0,Hi Israel! You received 2 points for this essa...,NaN
4,Wonderful College Preparatory Academy - Writin...,8195,6th Grade - Benchmark #1 (Radios Essay) - Fal...,Joie Garcia,2023-12-09 11:45:00,1092393,"Carranza, Zaiahs",2.0,2.0,Hi Zaiahs! You received 2 points for this essa...,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1159,Wonderful College Preparatory Academy - Writin...,8497,8th Grade - Benchmark #1 (Lewis and Clark Essa...,Joie Garcia,2023-12-12 14:02:00,1092549,"Sanchez, Angeline",2.0,2.0,Hi Angeline! You provide good context about th...,NaN
1160,Wonderful College Preparatory Academy - Writin...,8497,8th Grade - Benchmark #1 (Lewis and Clark Essa...,Joie Garcia,2023-12-12 14:02:00,1095783,"Sanchez, Jose",1.0,1.0,Hi Jose! You chose to include several importan...,NaN
1161,Wonderful College Preparatory Academy - Writin...,8497,8th Grade - Benchmark #1 (Lewis and Clark Essa...,Joie Garcia,2023-12-12 14:02:00,1092552,"Solorio Mendoza, Liset",1.0,1.0,Hello Liset! You demonstrate a good attempt at...,NaN
1162,Wonderful College Preparatory Academy - Writin...,8497,8th Grade - Benchmark #1 (Lewis and Clark Essa...,Joie Garcia,2023-12-12 14:02:00,1094466,"Vargas, Eloy",1.0,1.0,Hi Eloy! Much of your response comes from Wil...,NaN


In [4]:
overall = input('Pathway of raw Illuminate learning data (get rid of quotations):   ')
overall = pd.read_excel(overall)

Pathway of raw Illuminate learning data (get rid of quotations):   C:\Users\derek.castleman\Desktop\CFA Data\CFA Data 23-24\CFA 2\Marco Learning CFA\Results\CFAELAOverallRaw.xls
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


In [5]:
overall

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12,16,75%,Standard Met,3
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6,16,37.5%,Standard Not Met,1
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12,16,75%,Standard Met,3
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12,16,75%,Standard Met,3
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13,16,81.25%,Standard Exceeded,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1996,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,0,4,0%,Standard Not Met,1
1997,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,0,4,0%,Standard Not Met,1
1998,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,0,4,0%,Standard Not Met,1
1999,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,0,4,0%,Standard Not Met,1


In [6]:
overall_standards = input('Pathway of raw Illuminate Standards learning data (get rid of quotations):   ')
overall_standards = pd.read_excel(overall_standards)

Pathway of raw Illuminate Standards learning data (get rid of quotations):   C:\Users\derek.castleman\Desktop\CFA Data\CFA Data 23-24\CFA 2\Full ELA Standards Raw.xls


In [7]:
marco = marco[['student_id', 'Total']]
marco

,student_id,Total
0,1095032,3.0
1,1095456,NaN
2,1095921,3.0
3,1095465,2.0
4,1092393,2.0
...,...,...
1159,1092549,2.0
1160,1095783,1.0
1161,1092552,1.0
1162,1094466,1.0


In [8]:
marco = marco.fillna(0)
marco

,student_id,Total
0,1095032,3.0
1,1095456,0.0
2,1095921,3.0
3,1095465,2.0
4,1092393,2.0
...,...,...
1159,1092549,2.0
1160,1095783,1.0
1161,1092552,1.0
1162,1094466,1.0


In [9]:
combined = pd.merge(overall, marco, how='left', left_on='Student Id', right_on='student_id')
combined

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12,16,75%,Standard Met,3,NaN,NaN
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6,16,37.5%,Standard Not Met,1,NaN,NaN
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12,16,75%,Standard Met,3,NaN,NaN
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12,16,75%,Standard Met,3,NaN,NaN
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13,16,81.25%,Standard Exceeded,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,0,4,0%,Standard Not Met,1,1093014.0,1.0
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,0,4,0%,Standard Not Met,1,1095530.0,3.0
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,0,4,0%,Standard Not Met,1,1094311.0,4.0
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,0,4,0%,Standard Not Met,1,1093012.0,4.0


In [10]:
combined = combined.drop_duplicates(subset=['Student Id'])
combined

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12,16,75%,Standard Met,3,NaN,NaN
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6,16,37.5%,Standard Not Met,1,NaN,NaN
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12,16,75%,Standard Met,3,NaN,NaN
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12,16,75%,Standard Met,3,NaN,NaN
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13,16,81.25%,Standard Exceeded,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,0,4,0%,Standard Not Met,1,1093014.0,1.0
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,0,4,0%,Standard Not Met,1,1095530.0,3.0
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,0,4,0%,Standard Not Met,1,1094311.0,4.0
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,0,4,0%,Standard Not Met,1,1093012.0,4.0


In [11]:
combined['Number Correct'] = combined.apply(lambda row: row['Total']
                            if pd.notna(row['Total']) else row['Number Correct'], axis=1)
combined

C:\Users\DEREK~1.CAS\AppData\Local\Temp/ipykernel_13516/41184462.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['Number Correct'] = combined.apply(lambda row: row['Total']


,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75%,Standard Met,3,NaN,NaN
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1,NaN,NaN
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75%,Standard Met,3,NaN,NaN
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75%,Standard Met,3,NaN,NaN
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,0%,Standard Not Met,1,1093014.0,1.0
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,0%,Standard Not Met,1,1095530.0,3.0
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,0%,Standard Not Met,1,1094311.0,4.0
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,0%,Standard Not Met,1,1093012.0,4.0


In [12]:
combined['Performance Band Level'] = combined.apply(lambda row: row['Total']
                            if pd.notna(row['Total']) else row['Performance Band Level'], axis=1)
combined

C:\Users\DEREK~1.CAS\AppData\Local\Temp/ipykernel_13516/471894882.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['Performance Band Level'] = combined.apply(lambda row: row['Total']


,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75%,Standard Met,3.0,NaN,NaN
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1.0,NaN,NaN
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75%,Standard Met,3.0,NaN,NaN
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75%,Standard Met,3.0,NaN,NaN
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,0%,Standard Not Met,1.0,1093014.0,1.0
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,0%,Standard Not Met,3.0,1095530.0,3.0
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,0%,Standard Not Met,4.0,1094311.0,4.0
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,0%,Standard Not Met,4.0,1093012.0,4.0


In [13]:
combined['Percentage'] = ((combined['Number Correct'] / combined['Number Possible']) * 100).round(2).astype(str) + '%'
combined

C:\Users\DEREK~1.CAS\AppData\Local\Temp/ipykernel_13516/2504597019.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['Percentage'] = ((combined['Number Correct'] / combined['Number Possible']) * 100).round(2).astype(str) + '%'


,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total,Percentage
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75%,Standard Met,3.0,NaN,NaN,75.0%
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1.0,NaN,NaN,37.5%
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75%,Standard Met,3.0,NaN,NaN,75.0%
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75%,Standard Met,3.0,NaN,NaN,75.0%
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4.0,NaN,NaN,81.25%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,0%,Standard Not Met,1.0,1093014.0,1.0,25.0%
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,0%,Standard Not Met,3.0,1095530.0,3.0,75.0%
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,0%,Standard Not Met,4.0,1094311.0,4.0,100.0%
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,0%,Standard Not Met,4.0,1093012.0,4.0,100.0%


In [14]:
combined['% Correct'] = combined['Percentage']
combined

C:\Users\DEREK~1.CAS\AppData\Local\Temp/ipykernel_13516/3537489763.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  combined['% Correct'] = combined['Percentage']


,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total,Percentage
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.0%
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1.0,NaN,NaN,37.5%
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.0%
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.0%
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4.0,NaN,NaN,81.25%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,25.0%,Standard Not Met,1.0,1093014.0,1.0,25.0%
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,75.0%,Standard Not Met,3.0,1095530.0,3.0,75.0%
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,100.0%,Standard Not Met,4.0,1094311.0,4.0,100.0%
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,100.0%,Standard Not Met,4.0,1093012.0,4.0,100.0%


In [15]:
columns_to_drop = ['student_id', 'Total', 'Percentage']
final_overall = combined.drop(columns=columns_to_drop)
final_overall

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75.0%,Standard Met,3.0
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1.0
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,25.0%,Standard Not Met,1.0
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,75.0%,Standard Not Met,3.0
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,100.0%,Standard Not Met,4.0
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,100.0%,Standard Not Met,4.0


In [16]:
overall_standards

,Current Site ID,Student ID,State Student ID,Current Grade Level,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,...,HS.ELA.G9.CFA#2. 23-24 CCSS.ELA-Literacy.W.9-10.2 Performance Band Level,HS.ELA.G9.CFA#2. 23-24 Subject Area,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Points Possible,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Percent Correct,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Performance Band Level,HS.ELA.G10.CFA#2.23-24 Subject Area,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Points Possible,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Percent Correct,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Performance Band Level,HS.ELA.G11.CFA#2.23-24 Subject Area
0,1,1094919,7.535043e+09,9,Aabdiel,Alvarez,R,Redesignated Fluent English Proficient,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,1094814,3.391156e+09,3,Aaden,Delgado-Zavala,E,English Only,T,12-11-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1093747,3.416446e+09,7,Aadyn,Fernandez,E,English Only,F,11-29-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1095428,6.269798e+09,2,Aaleyah,Sanchez,I,Initially Fluent English Proficient (I-FEP),F,11-30-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,1093462,6.053400e+09,9,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,2,1094774,9.472721e+09,7,Zoey,Flores,E,English Only,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2386,2,1095490,6.421104e+09,7,Zoey,Maldonado,E,English Only,F,11-30-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2387,6,1096083,1.632009e+09,TK,Zoey,Perez,E,English Only,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2388,4,1096124,7.306925e+09,K,Zooey,Camarena,E,English Only,F,12-12-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
combined

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total,Percentage
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.0%
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1.0,NaN,NaN,37.5%
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.0%
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.0%
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4.0,NaN,NaN,81.25%
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,25.0%,Standard Not Met,1.0,1093014.0,1.0,25.0%
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,75.0%,Standard Not Met,3.0,1095530.0,3.0,75.0%
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,100.0%,Standard Not Met,4.0,1094311.0,4.0,100.0%
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,100.0%,Standard Not Met,4.0,1093012.0,4.0,100.0%


In [18]:
columns_to_drop = ['Percentage']
combined = combined.drop(columns=columns_to_drop)

In [19]:
combined

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1.0,NaN,NaN
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,25.0%,Standard Not Met,1.0,1093014.0,1.0
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,75.0%,Standard Not Met,3.0,1095530.0,3.0
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,100.0%,Standard Not Met,4.0,1094311.0,4.0
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,100.0%,Standard Not Met,4.0,1093012.0,4.0


In [20]:
combined['Percentage'] = ((combined['Number Correct'] / combined['Number Possible']) * 100).round(2)
combined

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total,Percentage
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.00
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.5%,Standard Not Met,1.0,NaN,NaN,37.50
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.00
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75.0%,Standard Met,3.0,NaN,NaN,75.00
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25%,Standard Exceeded,4.0,NaN,NaN,81.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,25.0%,Standard Not Met,1.0,1093014.0,1.0,25.00
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,75.0%,Standard Not Met,3.0,1095530.0,3.0,75.00
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,100.0%,Standard Not Met,4.0,1094311.0,4.0,100.00
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,100.0%,Standard Not Met,4.0,1093012.0,4.0,100.00


In [21]:
combined['% Correct'] = combined['Percentage']
combined

,Assessment,Student Id,SSID,First Name,Last Name,Birth Date,Grade Level,Date Taken,Number Correct,Number Possible,% Correct,Performance Band,Performance Band Level,student_id,Total,Percentage
0,ELEM.ELA.G1.CFA#2.23-24,1096177,3.240176e+09,Willow,Douglass-Bilvado,11/13/2016,1st Grade,12/12/2023,12.0,16,75.00,Standard Met,3.0,NaN,NaN,75.00
1,ELEM.ELA.G1.CFA#2.23-24,1096036,7.561458e+09,Eileen,Vazquez Aguilar,01/02/2017,1st Grade,12/13/2023,6.0,16,37.50,Standard Not Met,1.0,NaN,NaN,37.50
2,ELEM.ELA.G1.CFA#2.23-24,1095774,7.446021e+09,Alan,Chavez Negrete,02/16/2017,1st Grade,12/13/2023,12.0,16,75.00,Standard Met,3.0,NaN,NaN,75.00
3,ELEM.ELA.G1.CFA#2.23-24,1095753,2.673697e+09,Levi,Lopez,09/18/2016,1st Grade,12/13/2023,12.0,16,75.00,Standard Met,3.0,NaN,NaN,75.00
4,ELEM.ELA.G1.CFA#2.23-24,1095815,9.088117e+09,Leslie,Meza,12/26/2016,1st Grade,12/13/2023,13.0,16,81.25,Standard Exceeded,4.0,NaN,NaN,81.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,MS.ELA.G8.CFA#2.23-24,1093014,5.158756e+09,Matthew,Padilla,11/30/2009,8th Grade,11/30/2023,1.0,4,25.00,Standard Not Met,1.0,1093014.0,1.0,25.00
1998,MS.ELA.G8.CFA#2.23-24,1095530,1.395734e+09,Isadora,Bravo Hernandez,09/02/2009,8th Grade,12/04/2023,3.0,4,75.00,Standard Not Met,3.0,1095530.0,3.0,75.00
1999,MS.ELA.G8.CFA#2.23-24,1094311,9.550207e+09,Delaney,Mendoza,08/28/2010,8th Grade,12/04/2023,4.0,4,100.00,Standard Not Met,4.0,1094311.0,4.0,100.00
2000,MS.ELA.G8.CFA#2.23-24,1093012,4.286084e+09,Andrew,Mendoza Ruelas,12/27/2009,8th Grade,11/30/2023,4.0,4,100.00,Standard Not Met,4.0,1093012.0,4.0,100.00


In [22]:
marco_standards = combined[['student_id', 'Performance Band Level', '% Correct']]
marco_standards

,student_id,Performance Band Level,% Correct
0,NaN,3.0,75.00
1,NaN,1.0,37.50
2,NaN,3.0,75.00
3,NaN,3.0,75.00
4,NaN,4.0,81.25
...,...,...,...
1997,1093014.0,1.0,25.00
1998,1095530.0,3.0,75.00
1999,1094311.0,4.0,100.00
2000,1093012.0,4.0,100.00


In [23]:
overall_standards

,Current Site ID,Student ID,State Student ID,Current Grade Level,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,...,HS.ELA.G9.CFA#2. 23-24 CCSS.ELA-Literacy.W.9-10.2 Performance Band Level,HS.ELA.G9.CFA#2. 23-24 Subject Area,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Points Possible,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Percent Correct,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Performance Band Level,HS.ELA.G10.CFA#2.23-24 Subject Area,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Points Possible,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Percent Correct,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Performance Band Level,HS.ELA.G11.CFA#2.23-24 Subject Area
0,1,1094919,7.535043e+09,9,Aabdiel,Alvarez,R,Redesignated Fluent English Proficient,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,1094814,3.391156e+09,3,Aaden,Delgado-Zavala,E,English Only,T,12-11-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1093747,3.416446e+09,7,Aadyn,Fernandez,E,English Only,F,11-29-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,1095428,6.269798e+09,2,Aaleyah,Sanchez,I,Initially Fluent English Proficient (I-FEP),F,11-30-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,1093462,6.053400e+09,9,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,2,1094774,9.472721e+09,7,Zoey,Flores,E,English Only,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2386,2,1095490,6.421104e+09,7,Zoey,Maldonado,E,English Only,F,11-30-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2387,6,1096083,1.632009e+09,TK,Zoey,Perez,E,English Only,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2388,4,1096124,7.306925e+09,K,Zooey,Camarena,E,English Only,F,12-12-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
combined_standards = pd.merge(overall_standards, marco_standards, how='left', left_on='Student ID', right_on='student_id')
combined_standards

,Current Site ID,Student ID,State Student ID,Current Grade Level,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,...,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Percent Correct,HS.ELA.G10.CFA#2.23-24 CCSS.ELA-Literacy.W.9-10.2 Performance Band Level,HS.ELA.G10.CFA#2.23-24 Subject Area,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Points Possible,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Percent Correct,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-12.2 Performance Band Level,HS.ELA.G11.CFA#2.23-24 Subject Area,student_id,Performance Band Level_y,% Correct
0,1,1094919,7.535043e+09,9,Aabdiel,Alvarez,R,Redesignated Fluent English Proficient,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4,1094814,3.391156e+09,3,Aaden,Delgado-Zavala,E,English Only,T,12-11-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,1093747,3.416446e+09,7,Aadyn,Fernandez,E,English Only,F,11-29-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1093747.0,2.0,50.0
3,4,1095428,6.269798e+09,2,Aaleyah,Sanchez,I,Initially Fluent English Proficient (I-FEP),F,11-30-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8,1093462,6.053400e+09,9,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1093462.0,2.0,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2385,2,1094774,9.472721e+09,7,Zoey,Flores,E,English Only,T,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2386,2,1095490,6.421104e+09,7,Zoey,Maldonado,E,English Only,F,11-30-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1095490.0,2.0,50.0
2387,6,1096083,1.632009e+09,TK,Zoey,Perez,E,English Only,F,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2388,4,1096124,7.306925e+09,K,Zooey,Camarena,E,English Only,F,12-12-2023,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
melted = combined_standards.melt(['Current Site ID','Student ID', 'Current Grade Level', 'State Student ID',
             'First Name', 'Last Name', 'English Proficiency Code',
            'English Proficiency', 'Student Is Special Ed?', 'Date Taken', 'student_id',
            'Performance Band Level_y', '% Correct'], var_name='Standard', value_name='Level')
melted

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
0,1,1094919,9,7.535043e+09,Aabdiel,Alvarez,R,Redesignated Fluent English Proficient,T,NaN,NaN,NaN,NaN,Points,NaN
1,4,1094814,3,3.391156e+09,Aaden,Delgado-Zavala,E,English Only,T,12-11-2023,NaN,NaN,NaN,Points,1.0
2,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Points,0.0
3,4,1095428,2,6.269798e+09,Aaleyah,Sanchez,I,Initially Fluent English Proficient (I-FEP),F,11-30-2023,NaN,NaN,NaN,Points,16.0
4,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Points,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274845,2,1094774,7,9.472721e+09,Zoey,Flores,E,English Only,T,NaN,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274846,2,1095490,7,6.421104e+09,Zoey,Maldonado,E,English Only,F,11-30-2023,1095490.0,2.0,50.0,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274847,6,1096083,TK,1.632009e+09,Zoey,Perez,E,English Only,F,NaN,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274848,4,1096124,K,7.306925e+09,Zooey,Camarena,E,English Only,F,12-12-2023,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN


In [26]:
non_marco = melted[melted['student_id'].isna()]
non_marco

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
0,1,1094919,9,7.535043e+09,Aabdiel,Alvarez,R,Redesignated Fluent English Proficient,T,NaN,NaN,NaN,NaN,Points,NaN
1,4,1094814,3,3.391156e+09,Aaden,Delgado-Zavala,E,English Only,T,12-11-2023,NaN,NaN,NaN,Points,1.0
3,4,1095428,2,6.269798e+09,Aaleyah,Sanchez,I,Initially Fluent English Proficient (I-FEP),F,11-30-2023,NaN,NaN,NaN,Points,16.0
5,4,1096195,1,4.353682e+09,Aaliyah,Carr,E,English Only,F,12-05-2023,NaN,NaN,NaN,Points,14.0
6,4,1095715,K,6.517637e+09,Aaliyah,Cortes,E,English Only,F,12-06-2023,NaN,NaN,NaN,Points,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274844,4,1095345,2,8.255408e+09,Zoey,Ayon,E,English Only,F,11-30-2023,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274845,2,1094774,7,9.472721e+09,Zoey,Flores,E,English Only,T,NaN,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274847,6,1096083,TK,1.632009e+09,Zoey,Perez,E,English Only,F,NaN,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274848,4,1096124,K,7.306925e+09,Zooey,Camarena,E,English Only,F,12-12-2023,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN


In [27]:
marco_students = melted[melted['student_id'].notna()]
marco_students

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
2,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Points,0.0
4,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Points,0.0
9,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Points,0.0
11,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Points,0.0
14,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Points,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274832,1,1096028,9,6.746077e+09,Zachary,Hillman,E,English Only,F,12-04-2023,1096028.0,2.0,50.0,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274834,2,1092393,6,8.756169e+09,Zaiahs,Carranza,E,English Only,F,11-30-2023,1092393.0,2.0,50.0,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274840,2,1095523,8,7.326491e+09,Zlatan,Arteaga,R,Redesignated Fluent English Proficient,F,12-04-2023,1095523.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274842,2,1095670,7,6.099611e+09,Zoe,Alaniz Molina,R,Redesignated Fluent English Proficient,F,12-05-2023,1095670.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN


In [28]:
marco_students = marco_students[marco_students['Level'].notna()]
marco_students

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
2,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Points,0.0
4,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Points,0.0
9,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Points,0.0
11,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Points,0.0
14,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Points,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272352,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0
272357,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0
272377,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0
272412,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0


In [29]:
performance_band = marco_students[marco_students['Standard'].str.contains('Performance Band Level')]
performance_band

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
7172,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Performance Band Level_x,1.0
7174,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Performance Band Level_x,1.0
7179,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Performance Band Level_x,1.0
7181,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Performance Band Level_x,1.0
7184,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Performance Band Level_x,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272352,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0
272357,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0
272377,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0
272412,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,1.0


In [30]:
performance_band['Level'] = performance_band['Performance Band Level_y']
performance_band

C:\Users\DEREK~1.CAS\AppData\Local\Temp/ipykernel_13516/121662488.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  performance_band['Level'] = performance_band['Performance Band Level_y']


,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
7172,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Performance Band Level_x,2.0
7174,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Performance Band Level_x,2.0
7179,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Performance Band Level_x,3.0
7181,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Performance Band Level_x,3.0
7184,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Performance Band Level_x,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272352,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,3.0
272357,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,3.0
272377,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
272412,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0


In [31]:
percent_correct = marco_students[marco_students['Standard'].str.contains('Percent Correct')]
percent_correct

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
4782,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Percent Correct,0.0
4784,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Percent Correct,0.0
4789,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Percent Correct,0.0
4791,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Percent Correct,0.0
4794,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Percent Correct,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269962,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
269967,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
269987,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
270022,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0


In [32]:
percent_correct['Level'] = percent_correct['% Correct']
percent_correct

C:\Users\DEREK~1.CAS\AppData\Local\Temp/ipykernel_13516/3103148790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  percent_correct['Level'] = percent_correct['% Correct']


,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
4782,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Percent Correct,50.0
4784,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Percent Correct,50.0
4789,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Percent Correct,75.0
4791,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Percent Correct,75.0
4794,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Percent Correct,50.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269962,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,75.0
269967,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,75.0
269987,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
270022,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0


In [33]:
mask = marco_students['Standard'].str.contains('Performance Band Level')
marco_students = marco_students[~mask]

In [34]:
marco_students

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
2,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Points,0.0
4,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Points,0.0
9,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Points,0.0
11,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Points,0.0
14,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Points,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269962,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
269967,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
269987,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
270022,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0


In [35]:
mask = marco_students['Standard'].str.contains('Percent Correct')
marco_students = marco_students[~mask]
marco_students

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
2,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Points,0.0
4,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Points,0.0
9,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Points,0.0
11,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Points,0.0
14,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Points,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267572,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,4.0
267577,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,4.0
267597,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,4.0
267632,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,4.0


In [36]:
marco_students = pd.concat([marco_students, performance_band, percent_correct])
marco_students

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
2,2,1093747,7,3.416446e+09,Aadyn,Fernandez,E,English Only,F,11-29-2023,1093747.0,2.0,50.0,Points,0.0
4,8,1093462,9,6.053400e+09,Aaliyah,Avila,R,Redesignated Fluent English Proficient,F,11-29-2023,1093462.0,2.0,50.0,Points,0.0
9,2,1094697,7,7.216499e+09,Aaliyah,Lango Garibay,E,English Only,F,11-30-2023,1094697.0,3.0,75.0,Points,0.0
11,1,1093114,10,1.275548e+09,Aaron,Espinoza,R,Redesignated Fluent English Proficient,F,12-04-2023,1093114.0,3.0,75.0,Points,0.0
14,8,1093657,11,8.278195e+09,Aaron,Lopez Rondan,R,Redesignated Fluent English Proficient,F,12-01-2023,1093657.0,2.0,50.0,Points,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269962,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,75.0
269967,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,75.0
269987,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
270022,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0


In [37]:
non_marco

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
0,1,1094919,9,7.535043e+09,Aabdiel,Alvarez,R,Redesignated Fluent English Proficient,T,NaN,NaN,NaN,NaN,Points,NaN
1,4,1094814,3,3.391156e+09,Aaden,Delgado-Zavala,E,English Only,T,12-11-2023,NaN,NaN,NaN,Points,1.0
3,4,1095428,2,6.269798e+09,Aaleyah,Sanchez,I,Initially Fluent English Proficient (I-FEP),F,11-30-2023,NaN,NaN,NaN,Points,16.0
5,4,1096195,1,4.353682e+09,Aaliyah,Carr,E,English Only,F,12-05-2023,NaN,NaN,NaN,Points,14.0
6,4,1095715,K,6.517637e+09,Aaliyah,Cortes,E,English Only,F,12-06-2023,NaN,NaN,NaN,Points,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274844,4,1095345,2,8.255408e+09,Zoey,Ayon,E,English Only,F,11-30-2023,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274845,2,1094774,7,9.472721e+09,Zoey,Flores,E,English Only,T,NaN,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274847,6,1096083,TK,1.632009e+09,Zoey,Perez,E,English Only,F,NaN,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN
274848,4,1096124,K,7.306925e+09,Zooey,Camarena,E,English Only,F,12-12-2023,NaN,NaN,NaN,HS.ELA.G11.CFA#2.23-24 Subject Area,NaN


In [38]:
full_students = pd.concat([non_marco, marco_students])
full_students

,Current Site ID,Student ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,student_id,Performance Band Level_y,% Correct,Standard,Level
0,1,1094919,9,7.535043e+09,Aabdiel,Alvarez,R,Redesignated Fluent English Proficient,T,NaN,NaN,NaN,NaN,Points,NaN
1,4,1094814,3,3.391156e+09,Aaden,Delgado-Zavala,E,English Only,T,12-11-2023,NaN,NaN,NaN,Points,1.0
3,4,1095428,2,6.269798e+09,Aaleyah,Sanchez,I,Initially Fluent English Proficient (I-FEP),F,11-30-2023,NaN,NaN,NaN,Points,16.0
5,4,1096195,1,4.353682e+09,Aaliyah,Carr,E,English Only,F,12-05-2023,NaN,NaN,NaN,Points,14.0
6,4,1095715,K,6.517637e+09,Aaliyah,Cortes,E,English Only,F,12-06-2023,NaN,NaN,NaN,Points,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
269962,1,1093649,11,7.265031e+09,Vicki,Gutierrez,E,English Only,F,12-04-2023,1093649.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,75.0
269967,1,1095236,11,3.231727e+09,Victor,Manzo,R,Redesignated Fluent English Proficient,T,12-05-2023,1095236.0,3.0,75.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,75.0
269987,1,1095237,11,1.943642e+09,William,Ardon,L,English Learner,T,12-05-2023,1095237.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0
270022,1,1095102,11,1.223497e+09,Yanitzia,Vargas,R,Redesignated Fluent English Proficient,F,12-05-2023,1095102.0,0.0,0.0,HS.ELA.G11.CFA#2.23-24 CCSS.ELA-Literacy.W.11-...,0.0


In [39]:
columns_to_drop = ['student_id', 'Performance Band Level_y', '% Correct']
full_students = full_students.drop(columns=columns_to_drop)

In [40]:
unmelted = full_students.groupby(['Student ID', 'Current Site ID', 'Current Grade Level', 'State Student ID',
             'First Name', 'Last Name', 'English Proficiency Code',
            'English Proficiency', 'Student Is Special Ed?', 'Date Taken', 'Standard'])['Level'].max().unstack()
unmelted

,,,,,,,,,Standard,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1 Percent Correct,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1 Performance Band Level,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1 Points Possible,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1.g Percent Correct,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1.g Performance Band Level,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1.g Points Possible,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.2 Percent Correct,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.2 Performance Band Level,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.2 Points Possible,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.RI.1.1 Percent Correct,...,MS.ELA.G7.CFA#2.23-24 Subject Area,MS.ELA.G8.CFA#2.23-24 CCSS.ELA-Literacy.W.8.2 Percent Correct,MS.ELA.G8.CFA#2.23-24 CCSS.ELA-Literacy.W.8.2 Performance Band Level,MS.ELA.G8.CFA#2.23-24 CCSS.ELA-Literacy.W.8.2 Points Possible,MS.ELA.G8.CFA#2.23-24 Subject Area,Percent Correct,Performance Band Label,Performance Band Level_x,Points,Points Possible
Student ID,Current Site ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,,,,,,,,,,,,,,,,,,,,,
1092314,2,6,8.188746e+09,Isaias,Garcia,L,English Learner,T,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Standard Not Met,0.0,0.0,4.0
1092315,2,6,4.910305e+09,Daniel,Gonzalez Urrea,R,Redesignated Fluent English Proficient,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.0,Standard Not Met,2.0,0.0,4.0
1092319,2,6,8.758598e+09,Stephanie,Nunez Rodriguez,R,Redesignated Fluent English Proficient,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,75.0,Standard Not Met,3.0,0.0,4.0
1092327,2,6,9.472427e+09,Valentina,De Hoyos,R,Redesignated Fluent English Proficient,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,75.0,Standard Not Met,3.0,0.0,4.0
1092328,2,7,4.253032e+09,Derrick,Lacsina,E,English Only,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.0,Standard Not Met,2.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096363,2,6,1.652698e+09,Jason,Limon,E,English Only,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25.0,Standard Not Met,1.0,0.0,4.0
1096365,4,4,4.001548e+09,Deric,Garcia,L,English Learner,F,01-04-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Standard Not Met,1.0,0.0,10.0
1096366,2,6,3.149458e+09,Klarissa,Barreto,E,English Only,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Standard Not Met,0.0,0.0,4.0


In [41]:
unmelted = unmelted.rename(columns={'Performance Band Level_x': 'Performance Band Level'})
unmelted

,,,,,,,,,Standard,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1 Percent Correct,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1 Performance Band Level,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1 Points Possible,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1.g Percent Correct,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1.g Performance Band Level,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.1.g Points Possible,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.2 Percent Correct,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.2 Performance Band Level,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.L.1.2 Points Possible,ELEM.ELA.G1.CFA#2.23-24 CCSS.ELA-Literacy.RI.1.1 Percent Correct,...,MS.ELA.G7.CFA#2.23-24 Subject Area,MS.ELA.G8.CFA#2.23-24 CCSS.ELA-Literacy.W.8.2 Percent Correct,MS.ELA.G8.CFA#2.23-24 CCSS.ELA-Literacy.W.8.2 Performance Band Level,MS.ELA.G8.CFA#2.23-24 CCSS.ELA-Literacy.W.8.2 Points Possible,MS.ELA.G8.CFA#2.23-24 Subject Area,Percent Correct,Performance Band Label,Performance Band Level,Points,Points Possible
Student ID,Current Site ID,Current Grade Level,State Student ID,First Name,Last Name,English Proficiency Code,English Proficiency,Student Is Special Ed?,Date Taken,,,,,,,,,,,,,,,,,,,,,
1092314,2,6,8.188746e+09,Isaias,Garcia,L,English Learner,T,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Standard Not Met,0.0,0.0,4.0
1092315,2,6,4.910305e+09,Daniel,Gonzalez Urrea,R,Redesignated Fluent English Proficient,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.0,Standard Not Met,2.0,0.0,4.0
1092319,2,6,8.758598e+09,Stephanie,Nunez Rodriguez,R,Redesignated Fluent English Proficient,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,75.0,Standard Not Met,3.0,0.0,4.0
1092327,2,6,9.472427e+09,Valentina,De Hoyos,R,Redesignated Fluent English Proficient,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,75.0,Standard Not Met,3.0,0.0,4.0
1092328,2,7,4.253032e+09,Derrick,Lacsina,E,English Only,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,50.0,Standard Not Met,2.0,0.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096363,2,6,1.652698e+09,Jason,Limon,E,English Only,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,25.0,Standard Not Met,1.0,0.0,4.0
1096365,4,4,4.001548e+09,Deric,Garcia,L,English Learner,F,01-04-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Standard Not Met,1.0,0.0,10.0
1096366,2,6,3.149458e+09,Klarissa,Barreto,E,English Only,F,11-30-2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,Standard Not Met,0.0,0.0,4.0


In [63]:
writer = pd.ExcelWriter(overall_final)
final_overall.to_excel(writer, sheet_name='Overall', index=False)
writer.save()              

ValueError: No engine for filetype: ''

In [53]:
writer = pd.ExcelWriter(standards_final)
unmelted.to_excel(writer, sheet_name='Overall', index=True)
writer.save()     